# PDF to Excel

In [10]:
import pdfplumber
import pandas as pd
import numpy as np

In [ ]:


def extract_and_process_table(pdf_path, output_excel):
    all_data = []
    
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # Extract tables from each page
            tables = page.extract_tables()
            for table in tables:
                if len(table) > 0:
                    for row in table:
                        # Clean and process the row
                        cleaned_row = [cell.strip() if cell else '' for cell in row]
                        all_data.append(cleaned_row)
    print(all_data)
    # Convert to DataFrame
    df = pd.DataFrame(all_data)
    
    # Post-process DataFrame to handle multi-line headers or other issues
    # Here you can add your logic to clean and align columns correctly
    # For example:
    df.columns = df.iloc[0]
    df = df.drop(0)
    df = df.reset_index(drop=True)
    
    # Export to Excel
    df.to_excel(output_excel, index=False)


In [11]:
pdf_path = "../DEL CARMEN GRAL.130.pdf"
all_data = []
    
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        # Extract tables from each page
        tables = page.extract_tables()
        for table in tables:
            if len(table) > 0:
                for row in table:
                    # Clean and process the row
                    cleaned_row = [cell.strip() if cell else '' for cell in row]
                    all_data.append(cleaned_row)

In [27]:
# Convert to DataFrame
df = pd.DataFrame(all_data)


# # Post-process DataFrame to handle multi-line headers or other issues
# # Here you can add your logic to clean and align columns correctly
# # For example:
df.columns = df.iloc[0]
df = df.drop(0)
df = df.reset_index(drop=True)


,ARTÍCULO,PRECIO\nPACK,CANT. X\nPACK,PRECIO\nUNITARIO
0,CERVEZAS,,,
1,AMSTEL LATA LAGER 473cc.,"$\n18.400,00",24,"$ 766,67"
2,ANDES LATA IPA 473cc.,"$\n24.700,00",24,"$ 1.029,17"
3,ANDES LATA NEGRA 473cc.,"$\n24.700,00",24,"$ 1.029,17"
4,ANDES LATA ROJA 473cc.,"$\n24.700,00",24,"$ 1.029,17"


In [31]:
new_name = [name.replace("\n", " ") for name in df.columns]

df.columns = new_name
df.columns

Index(['ARTÍCULO', 'PRECIO PACK', 'CANT. X PACK', 'PRECIO UNITARIO'], dtype='object')

In [175]:
output_path = "./pdf_converted.xlsx"
df.to_excel(output_path, index=False)

## More processing

> Final file: pdf_cleaned.xlsx

In [250]:
data = "./final excel/pdf_cleaned.xlsx"
df2 = pd.read_excel(data)
df2.head()

,ARTÍCULO,PRECIO PACK,CANT. X PACK,PRECIO UNITARIO
0,CERVEZAS,NaN,NaN,NaN
1,AMSTEL LATA LAGER 473cc.,"$\n18.400,00",24.0,"$ 766,67"
2,ANDES LATA IPA 473cc.,"$\n24.700,00",24.0,"$ 1.029,17"
3,ANDES LATA NEGRA 473cc.,"$\n24.700,00",24.0,"$ 1.029,17"
4,ANDES LATA ROJA 473cc.,"$\n24.700,00",24.0,"$ 1.029,17"


In [251]:
# convertir todo a str, incluido whitespace
df2["PRECIO PACK"] = df["PRECIO PACK"].astype(str)

# Sacar el \n a los valores
df2["PRECIO PACK"] = df2["PRECIO PACK"].apply(lambda x: x.replace("\n", ""))

# poner NaN values en los whitespaces (forma correcta de asignar valores)
mask = df2["PRECIO PACK"] == ""
df2.loc[mask, "PRECIO PACK"] = np.nan
# now it correctly shows the nan values
print(f"Nan: {df2['PRECIO PACK'].isna().sum()}\n")

# Crear nueva columna con los valores casteados a float
precio_float = df2["PRECIO PACK"].str.replace("$", "").str.replace(",", "")
# replace any empty string with NaN to correctly cast 
precio_float.replace("", np.nan)
# convert to float 
precio_float = pd.to_numeric(precio_float, errors='coerce')
# 
precio_float = precio_float * 1000

Nan: 9



In [252]:
# convertir todo a str, incluido whitespace
df2["PRECIO UNITARIO"] = df["PRECIO UNITARIO"].astype(str)

# Sacar el \n a los valores
df2["PRECIO UNITARIO"] = df2["PRECIO UNITARIO"].apply(lambda x: x.replace("\n", ""))

# poner NaN values en los whitespaces (forma correcta de asignar valores)
mask = df2["PRECIO UNITARIO"] == ""
df2.loc[mask, "PRECIO UNITARIO"] = np.nan
# now it correctly shows the nan values
print(f"Nan: {df2['PRECIO UNITARIO'].isna().sum()}\n")

# Crear nueva columna con los valores casteados a float
precio_float = df2["PRECIO UNITARIO"].str.replace("$", "").str.replace(",", "")
# replace any empty string with NaN to correctly cast 
precio_float.replace("", np.nan)
# convert to float 
precio_float = pd.to_numeric(precio_float, errors='coerce')
# 
precio_float = round(precio_float / 100, 2)
precio_float

Nan: 14



0         NaN
1      766.67
2        0.01
3        0.01
4        0.01
        ...  
576    665.00
577      0.02
578    658.33
579    658.33
580    900.00
Name: PRECIO UNITARIO, Length: 581, dtype: float64

In [253]:
df2.insert(loc=2, column="PRECIO PACK (float)", value=precio_float)
df2.insert(loc=5, column="PRECIO UNITARIO (float)", value=precio_float)


In [254]:
df2

,ARTÍCULO,PRECIO PACK,PRECIO PACK (float),CANT. X PACK,PRECIO UNITARIO,PRECIO UNITARIO (float)
0,CERVEZAS,NaN,NaN,NaN,NaN,NaN
1,AMSTEL LATA LAGER 473cc.,"$18.400,00",766.67,24.0,"$ 766,67",766.67
2,ANDES LATA IPA 473cc.,"$24.700,00",0.01,24.0,"$ 1.029,17",0.01
3,ANDES LATA NEGRA 473cc.,"$24.700,00",0.01,24.0,"$ 1.029,17",0.01
4,ANDES LATA ROJA 473cc.,"$24.700,00",0.01,24.0,"$ 1.029,17",0.01
...,...,...,...,...,...,...
576,QUILMES DESTAPADOR QUILMES ACERO,"$ 3.990,00",665.00,1.0,"$ 665,00",665.00
577,STANLEY MATE TERMICO ACERO INOXIDABLE 236ml. N...,"$11.850,00",0.02,1.0,"$ 1.975,00",0.02
578,ALCOHOL,"$ 3.950,00",658.33,NaN,"$ 658,33",658.33
579,ALGABO ALCOHOL EN GEL 300ml. DOSIFICADOR,"$ 3.950,00",658.33,12.0,"$ 658,33",658.33


In [225]:
df2.to_excel("./final excel/precios_final.xlsx", index=False)